# Lesson 05 Walkthrough: SQL Aggregations & GROUP BY

## Database Applications Development

**Welcome!** This walkthrough demonstrates SQL aggregate functions and grouping with the NBA dataset.

### What You'll Learn:
1. Aggregate functions: COUNT, SUM, AVG, MIN, MAX
2. GROUP BY for category analysis
3. HAVING to filter groups
4. Exporting results to Excel

### Prerequisites:
- ✅ Completed Lessons 01-04
- ✅ NBA database created
- ✅ Understand basic SQL SELECT queries

Let's analyze some NBA data!

Sure — here’s a **clean Markdown data dictionary** you can paste right into your notebook.

---

# NBA SQLite Data Dictionary

## Tables

* `players`
* `teams`
* `player_season_stats`
* `team_game_stats`

## Keys and Relationships

* `players.player_id` → referenced by `player_season_stats.player_id`
* `teams.team_id` → referenced by `player_season_stats.team_id` and `team_game_stats.team_id`
* `team_game_stats` uses a **composite primary key**: (`season`, `game_id`, `team_id`)
* `player_season_stats` uses a **composite primary key**: (`season`, `player_id`, `team_id`)

---

## `players`

**Description:** One row per player.

| Column      | Type    | Key | Description              |
| ----------- | ------- | --- | ------------------------ |
| `player_id` | INTEGER | PK  | Unique player identifier |
| `full_name` | TEXT    |     | Player full name         |

---

## `teams`

**Description:** One row per NBA team.

| Column         | Type    | Key | Description                                   |
| -------------- | ------- | --- | --------------------------------------------- |
| `team_id`      | INTEGER | PK  | Unique team identifier                        |
| `full_name`    | TEXT    |     | Team full name (e.g., “Cleveland Cavaliers”)  |
| `abbreviation` | TEXT    |     | Team abbreviation (e.g., “CLE”)               |
| `nickname`     | TEXT    |     | Team nickname/mascot name (e.g., “Cavaliers”) |
| `city`         | TEXT    |     | Team city                                     |
| `state`        | TEXT    |     | Team state                                    |
| `year_founded` | INTEGER |     | Year the franchise was founded                |

---

## `player_season_stats`

**Description:** Per-player season stats (one row per `player_id` + `team_id` + `season`).

| Column      | Type    | Key      | Description                                                   |
| ----------- | ------- | -------- | ------------------------------------------------------------- |
| `season`    | TEXT    | PK*      | Season label (format depends on your source, e.g., `2019-20`) |
| `player_id` | INTEGER | PK* / FK | Player ID → `players.player_id`                               |
| `team_id`   | INTEGER | PK* / FK | Team ID → `teams.team_id`                                     |
| `gp`        | INTEGER |          | Games played                                                  |
| `min`       | REAL    |          | Minutes per game (often per-game average)                     |
| `pts`       | REAL    |          | Points per game                                               |
| `reb`       | REAL    |          | Rebounds per game                                             |
| `ast`       | REAL    |          | Assists per game                                              |
| `stl`       | REAL    |          | Steals per game                                               |
| `blk`       | REAL    |          | Blocks per game                                               |
| `tov`       | REAL    |          | Turnovers per game                                            |
| `fg_pct`    | REAL    |          | Field goal percentage (0–1 or 0–100 depending on source)      |
| `fg3_pct`   | REAL    |          | 3-point percentage (0–1 or 0–100 depending on source)         |
| `ft_pct`    | REAL    |          | Free throw percentage (0–1 or 0–100 depending on source)      |

*Composite PK: (`season`, `player_id`, `team_id`)

---

## `team_game_stats`

**Description:** Team box score stats per game (one row per team per game per season).

| Column       | Type    | Key      | Description                                                        |
| ------------ | ------- | -------- | ------------------------------------------------------------------ |
| `season`     | TEXT    | PK*      | Season label                                                       |
| `game_id`    | TEXT    | PK*      | Unique game identifier                                             |
| `team_id`    | INTEGER | PK* / FK | Team ID → `teams.team_id`                                          |
| `game_date`  | TEXT    |          | Game date (stored as text)                                         |
| `matchup`    | TEXT    |          | Matchup string (e.g., `CLE vs. BOS` or `CLE @ BOS`)                |
| `wl`         | TEXT    |          | Win/Loss flag (`W` or `L`)                                         |
| `pts`        | INTEGER |          | Team points                                                        |
| `fgm`        | INTEGER |          | Field goals made                                                   |
| `fga`        | INTEGER |          | Field goals attempted                                              |
| `fg3m`       | INTEGER |          | 3PT field goals made                                               |
| `fg3a`       | INTEGER |          | 3PT field goals attempted                                          |
| `ftm`        | INTEGER |          | Free throws made                                                   |
| `fta`        | INTEGER |          | Free throws attempted                                              |
| `oreb`       | INTEGER |          | Offensive rebounds                                                 |
| `dreb`       | INTEGER |          | Defensive rebounds                                                 |
| `reb`        | INTEGER |          | Total rebounds                                                     |
| `ast`        | INTEGER |          | Assists                                                            |
| `stl`        | INTEGER |          | Steals                                                             |
| `blk`        | INTEGER |          | Blocks                                                             |
| `tov`        | INTEGER |          | Turnovers                                                          |
| `plus_minus` | INTEGER |          | Point differential while team was on court (as provided by source) |

*Composite PK: (`season`, `game_id`, `team_id`)

---

If you want, I can also add a **“Notes / Known Caveats”** section (super useful for students) that flags:

* `% columns are sometimes 0–1 vs 0–100`
* `game_date` stored as TEXT (and how to convert to real dates)
* whether `min/pts/etc.` are totals or per-game averages (we can confirm with one quick query)


---

## Setup - The tables in this dataset are already created - so we don't have to do any other setup besides connecting to the db. 

In [35]:
import pandas as pd
import sqlite3

# Connect to database
conn = sqlite3.connect('nba_5seasons.db')
print("✅ Connected to nba_5seasons.db")

✅ Connected to nba_5seasons.db


## Let's look at the tables we have in the database first: 
In SQLite, tables are stored as metadata in a system table called `sqlite_master`.

In [15]:
# SQL query to list all tables in the SQLite database
query = """
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
"""

# Run the query and load the results into a pandas DataFrame
tables = pd.read_sql(query, conn)

# Display the table names
tables

,name
0,player_season_stats
1,players
2,team_game_stats
3,teams


## As an example, let's look at one of the tables columns/fields by using the 

In [16]:
pd.read_sql("PRAGMA table_info(player_season_stats);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,season,TEXT,0,None,1
1,1,player_id,INTEGER,0,None,2
2,2,team_id,INTEGER,0,None,3
3,3,gp,INTEGER,0,None,0
4,4,min,REAL,0,None,0
5,5,pts,REAL,0,None,0
6,6,reb,REAL,0,None,0
7,7,ast,REAL,0,None,0
8,8,stl,REAL,0,None,0
9,9,blk,REAL,0,None,0


In [17]:
pd.read_sql("PRAGMA table_info(players);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,player_id,INTEGER,0,None,1
1,1,full_name,TEXT,0,None,0


In [18]:
pd.read_sql("PRAGMA table_info(team_game_stats);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,season,TEXT,0,None,1
1,1,game_id,TEXT,0,None,2
2,2,team_id,INTEGER,0,None,3
3,3,game_date,TEXT,0,None,0
4,4,matchup,TEXT,0,None,0
5,5,wl,TEXT,0,None,0
6,6,pts,INTEGER,0,None,0
7,7,fgm,INTEGER,0,None,0
8,8,fga,INTEGER,0,None,0
9,9,fg3m,INTEGER,0,None,0


In [19]:
pd.read_sql("PRAGMA table_info(teams);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,team_id,INTEGER,0,None,1
1,1,full_name,TEXT,0,None,0
2,2,abbreviation,TEXT,0,None,0
3,3,nickname,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,state,TEXT,0,None,0
6,6,year_founded,INTEGER,0,None,0


---

## Part 1: COUNT() - Counting Rows

COUNT() tells us how many rows match our criteria.

### Example 1.1: Count All Teams

In [36]:
# How many teams are in our database?
query = """
SELECT COUNT(*) as total_teams
FROM teams
"""

result = pd.read_sql(query, conn)
print("Total NBA Teams:")
display(result)

# COUNT(*) counts all rows in the table

Total NBA Teams:


,total_teams
0,30


### Example 1.2: Count Player Records

In [21]:
# How many player-season records for 2021-22?
query = """
SELECT COUNT(*) as player_records
FROM player_season_stats
WHERE season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("Player Season Records (2021-22):")
display(result)

# This counts how many players have stats recorded for that season

Player Season Records (2021-22):


,player_records
0,605


**Key Insight:** COUNT(*) counts all rows. COUNT(column) would only count non-NULL values in that column.

---

## Part 2: SUM() - Adding Values

SUM() adds up numeric values.

### Example 2.1: Lakers Total Points

In [22]:
# Total points scored by Lakers in 2021-22
query = """
SELECT SUM(pts) as total_points
FROM team_game_stats
WHERE team_id = 1610612747
  AND season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("Lakers Total Points (2021-22 Season):")
display(result)

# SUM adds up the pts column for all Lakers games

Lakers Total Points (2021-22 Season):


,total_points
0,9192


---

## Part 3: AVG() - Calculating Averages

AVG() calculates the mean value.

### Example 3.1: Warriors Average Points

In [23]:
# Warriors average points per game
query = """
SELECT ROUND(AVG(pts), 1) as avg_points_per_game
FROM team_game_stats
WHERE team_id = 1610612744
  AND season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("Warriors Average Points Per Game (2021-22):")
display(result)

# AVG calculates mean, ROUND formats to 1 decimal place

Warriors Average Points Per Game (2021-22):


,avg_points_per_game
0,111.0


---

## Part 4: MIN() and MAX() - Finding Extremes

MIN and MAX find the smallest and largest values.

### Example 4.1: Scoring Range

In [24]:
# Highest and lowest scores in any game (2021-22)
query = """
SELECT 
    MIN(pts) as lowest_score,
    MAX(pts) as highest_score,
    MAX(pts) - MIN(pts) as score_range
FROM team_game_stats
WHERE season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("Scoring Range for 2021-22 Season:")
display(result)

# Shows the extremes and calculates the range

Scoring Range for 2021-22 Season:


,lowest_score,highest_score,score_range
0,75,158,83


---

## Part 5: Combining Multiple Aggregates

You can use multiple aggregate functions in one query.

### Example 5.1: Complete Statistical Summary

In [25]:
# Comprehensive stats for all 2021-22 games
query = """
SELECT 
    COUNT(*) as total_games,
    SUM(pts) as total_points,
    ROUND(AVG(pts), 1) as avg_points,
    MIN(pts) as min_score,
    MAX(pts) as max_score
FROM team_game_stats
WHERE season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("2021-22 Season Statistics Summary:")
display(result)

# One query gives us a complete statistical overview

2021-22 Season Statistics Summary:


,total_games,total_points,avg_points,min_score,max_score
0,2460,272115,110.6,75,158


**Key Insight:** Combining aggregates gives a comprehensive summary in one query!

---

## Part 6: GROUP BY - Category Analysis

GROUP BY splits data into groups and aggregates each separately.

### Example 6.1: Games Per Team (Without GROUP BY)

In [26]:
# Without GROUP BY - just counts ALL games
query = """
SELECT COUNT(*) as total_games
FROM team_game_stats
WHERE season = '2021-22'
"""

result = pd.read_sql(query, conn)
print("Total games across all teams:")
display(result)

# This gives us one number for the entire dataset

Total games across all teams:


,total_games
0,2460


### Example 6.2: Games Per Team (With GROUP BY)

In [27]:
# With GROUP BY - counts games PER TEAM
query = """
SELECT 
    team_id,
    COUNT(*) as games_played
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
ORDER BY games_played DESC
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("Games played by each team (top 10):")
display(result)

# GROUP BY team_id creates separate groups for each team
# COUNT(*) runs on each group independently

Games played by each team (top 10):


,team_id,games_played
0,1610612766,82
1,1610612765,82
2,1610612764,82
3,1610612763,82
4,1610612762,82
5,1610612761,82
6,1610612760,82
7,1610612759,82
8,1610612758,82
9,1610612757,82


### Example 6.3: Add Team Names with JOIN

In [28]:
# Same query but with team names for readability
query = """
SELECT 
    t.full_name as team,
    t.city,
    COUNT(tgs.game_id) as games_played
FROM teams t
JOIN team_game_stats tgs ON t.team_id = tgs.team_id
WHERE tgs.season = '2021-22'
GROUP BY t.team_id, t.full_name, t.city
ORDER BY games_played DESC
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("Games Per Team with Names (2021-22):")
display(result)

# JOIN brings in team names
# GROUP BY includes all non-aggregated columns

Games Per Team with Names (2021-22):


,team,city,games_played
0,Atlanta Hawks,Atlanta,82
1,Boston Celtics,Boston,82
2,Cleveland Cavaliers,Cleveland,82
3,New Orleans Pelicans,New Orleans,82
4,Chicago Bulls,Chicago,82
5,Dallas Mavericks,Dallas,82
6,Denver Nuggets,Denver,82
7,Golden State Warriors,San Francisco,82
8,Houston Rockets,Houston,82
9,Los Angeles Clippers,Los Angeles,82


**Critical Rule:** Every column in SELECT that's NOT an aggregate function MUST be in GROUP BY!

---

## Part 7: HAVING - Filtering Groups

HAVING filters groups AFTER aggregation (WHERE filters rows BEFORE).

### Example 7.1: WHERE vs HAVING

In [29]:
# WHERE filters rows before grouping
query_where = """
SELECT 
    team_id,
    COUNT(*) as high_scoring_games
FROM team_game_stats
WHERE season = '2021-22'
  AND pts > 120
GROUP BY team_id
ORDER BY high_scoring_games DESC
LIMIT 5
"""

result_where = pd.read_sql(query_where, conn)
print("Teams with most 120+ point games (WHERE filters games first):")
display(result_where)

Teams with most 120+ point games (WHERE filters games first):


,team_id,high_scoring_games
0,1610612766,30
1,1610612750,30
2,1610612749,28
3,1610612737,28
4,1610612763,27


In [30]:
# HAVING filters groups after aggregating
query_having = """
SELECT 
    team_id,
    COUNT(*) as total_games,
    ROUND(AVG(pts), 1) as avg_points
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
HAVING AVG(pts) > 110
ORDER BY avg_points DESC
"""

result_having = pd.read_sql(query_having, conn)
print("Teams averaging 110+ PPG (HAVING filters groups):")
display(result_having)

# WHERE filters individual games (before grouping)
# HAVING filters teams by their average (after grouping)

Teams averaging 110+ PPG (HAVING filters groups):


,team_id,total_games,avg_points
0,1610612750,82,115.9
1,1610612763,82,115.6
2,1610612749,82,115.5
3,1610612766,82,115.3
4,1610612756,82,114.8
5,1610612737,82,113.9
6,1610612762,82,113.6
7,1610612759,82,113.2
8,1610612751,82,112.9
9,1610612743,82,112.7


**Key Difference:**
- WHERE: "Show me games where [condition]"
- HAVING: "Show me teams where their aggregate [condition]"

---

## Part 8: Exporting to Excel

Now let's export our results to Excel for further analysis.

### Example 8.1: Single Sheet Export

In [31]:
# Create a comprehensive team statistics query
query = """
SELECT 
    t.full_name as team,
    t.city,
    t.state,
    COUNT(tgs.game_id) as games_played,
    ROUND(AVG(tgs.pts), 1) as avg_points,
    MIN(tgs.pts) as season_low,
    MAX(tgs.pts) as season_high
FROM teams t
JOIN team_game_stats tgs ON t.team_id = tgs.team_id
WHERE tgs.season = '2021-22'
GROUP BY t.team_id, t.full_name, t.city, t.state
ORDER BY avg_points DESC
"""

# Execute query
team_stats = pd.read_sql(query, conn)

# Display what we're exporting
print("Team Performance Statistics:")
display(team_stats.head(10))

# Export to Excel
team_stats.to_excel('team_performance_2021_22.xlsx', index=False, sheet_name='Team Stats')
print("\n✅ Exported to team_performance_2021_22.xlsx")

Team Performance Statistics:


,team,city,state,games_played,avg_points,season_low,season_high
0,Minnesota Timberwolves,Minnesota,Minnesota,82,115.9,84,149
1,Memphis Grizzlies,Memphis,Tennessee,82,115.6,85,152
2,Milwaukee Bucks,Milwaukee,Wisconsin,82,115.5,90,142
3,Charlotte Hornets,Charlotte,North Carolina,82,115.3,86,158
4,Phoenix Suns,Phoenix,Arizona,82,114.8,95,140
5,Atlanta Hawks,Atlanta,Georgia,82,113.9,87,136
6,Utah Jazz,Utah,Utah,82,113.6,90,137
7,San Antonio Spurs,San Antonio,Texas,82,113.2,90,157
8,Brooklyn Nets,Brooklyn,New York,82,112.9,91,150
9,Denver Nuggets,Denver,Colorado,82,112.7,85,141



✅ Exported to team_performance_2021_22.xlsx


### Example 8.2: Multi-Sheet Export

In [32]:
# Create multiple related datasets

# Sheet 1: Some Basic Overall Stats
query_overall = """
SELECT
    t.full_name AS team,
    COUNT(tgs.game_id) AS games,
    SUM(tgs.pts) AS total_pts,
    SUM(tgs.fgm) AS total_fgm,
    SUM(tgs.fga) AS total_fga,
    SUM(tgs.fg3m) AS total_fg3m,
    SUM(tgs.fg3a) AS total_fg3a
FROM teams t
JOIN team_game_stats tgs ON t.team_id = tgs.team_id
WHERE tgs.season = '2021-22'
GROUP BY t.team_id, t.full_name;
"""
overall = pd.read_sql(query_overall, conn)


# Sheet 2: Win Records
query_wins = """
SELECT 
    t.full_name as team,
    SUM(CASE WHEN tgs.wl = 'W' THEN 1 ELSE 0 END) as wins,
    SUM(CASE WHEN tgs.wl = 'L' THEN 1 ELSE 0 END) as losses
FROM teams t
JOIN team_game_stats tgs ON t.team_id = tgs.team_id
WHERE tgs.season = '2021-22'
GROUP BY t.team_id, t.full_name
ORDER BY wins DESC
"""
wins = pd.read_sql(query_wins, conn)

# Sheet 3: Scoring Leaders
query_scoring = """
SELECT 
    team_id,
    MAX(pts) as best_game,
    MIN(pts) as worst_game,
    MAX(pts) - MIN(pts) as consistency
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
"""
scoring = pd.read_sql(query_scoring, conn)

# Export all sheets to one Excel file
with pd.ExcelWriter('nba_analysis_2021_22.xlsx', engine='openpyxl') as writer:
    overall.to_excel(writer, sheet_name='Overall Stats', index=False)
    wins.to_excel(writer, sheet_name='Win Records', index=False)
    scoring.to_excel(writer, sheet_name='Scoring Range', index=False)

print("\n✅ Exported to nba_analysis_2021_22.xlsx (3 sheets)")
print(f"   - Overall Stats: {len(overall)} teams")
print(f"   - Win Records: {len(wins)} teams")
print(f"   - Scoring Range: {len(scoring)} teams")


✅ Exported to nba_analysis_2021_22.xlsx (3 sheets)
   - Overall Stats: 30 teams
   - Win Records: 30 teams
   - Scoring Range: 30 teams


---

## Part 9: Complete Example - Team Consistency Analysis

Let's put it all together with a comprehensive analysis.

In [33]:
# Analyze scoring consistency across teams
# Focus: points consistency
# Shot totals are included so FG% can be calculated in Excel (not SQL)

query = """
SELECT 
    t.full_name AS team,
    COUNT(tgs.game_id) AS games,
    ROUND(AVG(tgs.pts), 1) AS avg_points,
    MIN(tgs.pts) AS lowest_game,
    MAX(tgs.pts) AS highest_game,
    (MAX(tgs.pts) - MIN(tgs.pts)) AS scoring_range,
    SUM(tgs.fgm) AS total_fgm,
    SUM(tgs.fga) AS total_fga
FROM teams t
JOIN team_game_stats tgs 
    ON t.team_id = tgs.team_id
WHERE tgs.season = '2021-22'
GROUP BY t.team_id, t.full_name
HAVING COUNT(tgs.game_id) >= 70
ORDER BY scoring_range ASC
LIMIT 15;
"""

# Run query and load into DataFrame
consistency = pd.read_sql(query, conn)

print("Most Consistent Teams (Smallest Scoring Range):")
display(consistency)

# Export results to Excel
consistency.to_excel(
    'team_consistency_2021_22.xlsx',
    index=False,
    sheet_name='Consistency'
)

print("\n✅ Exported to team_consistency_2021_22.xlsx")

# Notes:
# - scoring_range = MAX(points) - MIN(points)
# - smaller range = more consistent scoring
# - FG% should be calculated in Excel as: total_fgm / total_fga


Most Consistent Teams (Smallest Scoring Range):


,team,games,avg_points,lowest_game,highest_game,scoring_range,total_fgm,total_fga
0,New Orleans Pelicans,82,109.3,85,130,45,3294,7212
1,Phoenix Suns,82,114.8,95,140,45,3581,7389
2,Utah Jazz,82,113.6,90,137,47,3327,7067
3,Atlanta Hawks,82,113.9,87,136,49,3401,7241
4,Indiana Pacers,82,111.5,84,134,50,3398,7338
5,Orlando Magic,82,104.2,79,130,51,3139,7240
6,Milwaukee Bucks,82,115.5,90,142,52,3429,7331
7,Oklahoma City Thunder,82,103.7,79,134,55,3141,7310
8,Denver Nuggets,82,112.7,85,141,56,3416,7079
9,Golden State Warriors,82,111.0,82,138,56,3323,7087



✅ Exported to team_consistency_2021_22.xlsx


**Analysis Insight:** Teams with smaller scoring ranges are more predictable and consistent!

---

## Summary

You've learned how to:

✅ Use aggregate functions: COUNT, SUM, AVG, MIN, MAX  
✅ Group data with GROUP BY  
✅ Filter groups with HAVING  
✅ Combine aggregates for comprehensive statistics  
✅ Export query results to Excel (single and multi-sheet)  
✅ Understand WHERE (filter rows) vs HAVING (filter groups)  

### Key Patterns:
```sql
-- Pattern 1: Simple aggregation
SELECT COUNT(*), AVG(column) FROM table;

-- Pattern 2: Aggregation by category
SELECT category, COUNT(*), AVG(value)
FROM table
GROUP BY category;

-- Pattern 3: Filtered aggregation
SELECT category, COUNT(*) 
FROM table
WHERE condition
GROUP BY category
HAVING COUNT(*) > threshold;
```

### Next Steps:
- Complete the SQL task (write your own aggregate queries)
- Complete the Excel task (analyze the exported data)
- You're ready for Lesson 6: Database Design!

---

## Clean Up

In [34]:
# Close the database connection
conn.close()
print("✅ Connection closed")
print("Great work! You understand SQL aggregations!")

✅ Connection closed
Great work! You understand SQL aggregations!
